## Strava Data Analysis

This notebook join all the analysis made individually for the strava data as a final project for Practice: Software Development(Python) course. 

In [ ]:
#Check the functions available in the .py
import strava_analysis
dir(strava_analysis)


In [ ]:
# Import your functions
from strava_analysis import load_strava_data, plot_speed_distribution, extract_lat_lon, map_df, map_activities_colored_by_speed

### 1. Load the data
We use a function `load_strava_data()` that selects relevant columns, parses dates, and removes incomplete rows.


In [ ]:
#Use first function, upload the data
df = load_strava_data("../LeafletUpdates/nicole_strava.csv")
df

### 2. Visualize Speed
The function `plot_speed_distribution()` compares the average and max speeds using histograms.


In [ ]:
# plot speed distribution
plot_speed_distribution(df)

### 3.  Extract latitude and longitude

With `extract_lat_lon()`, we extract in two columns for each couple of coordenates separated information.

In [ ]:
df['start_lat'], df['start_lon'] = zip(*df['start_latlng'].map(extract_lat_lon))
df['end_lat'], df['end_lon'] = zip(*df['end_latlng'].map(extract_lat_lon))
df

### 4. Map the Activity starts and end points
With `map_df()`, we visualize where each activity began using folium.


In [ ]:
#Show activity start points
fmap = map_df(df)
fmap

### 5. Map of average speead start points
With `map_activities_colored_by_speed()`, we visualize the activity start point color-coded by average speed.

In [ ]:
speed_map = map_activities_colored_by_speed(df)
speed_map

# More geostatistical analysis

In [ ]:
from strava_analysis import (
    load_strava_data,
    advanced_hotspot_analysis,
    spatial_clustering_analysis,
    create_activity_locations_plot,
    create_hotspots_plot,
    create_performance_distributions_plot,
    create_temporal_patterns_plot,
    create_correlation_heatmap,
    create_distance_speed_scatter,
    create_clusters_plot,
    create_interactive_map,
    create_dashboard_html,
    display_html_map_simple,
    generate_comprehensive_report,
    main_analysis
)

In [ ]:

csv_file = "../LeafletUpdates/nicole_strava.csv"

results = main_analysis(csv_file)

gdf, hotspots, clusters = results
print(f"\nQuick summary:")
print(f"• Analyzed {len(gdf):,} activities")
print(f"• Total distance: {gdf['distance_km'].sum():,.0f} km")
print(f"• Average speed: {gdf['speed_kmh'].mean():.1f} km/h")
if clusters is not None and len(clusters) > 0:
    print(f"• found {len(clusters)} spatial clusters")

print("\nall visualizations are now separate files!")
print("interactive map displayed above (if supported)")
print("open strava_dashboard.html for a beautiful overview!")

In [ ]:
from IPython.display import HTML
import base64

with open('strava_interactive_map.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

iframe_html = f'''
<iframe srcdoc="{html_content.replace('"', '&quot;')}"
        width="100%" height="600" frameborder="0">
</iframe>
'''
HTML(iframe_html)

In [ ]:
with open('strava_dashboard.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

iframe_html = f'''
<iframe srcdoc="{html_content.replace('"', '&quot;')}"
        width="100%" height="600" frameborder="0">
</iframe>
'''
HTML(iframe_html)

# Creation of the speed and frequency heatmap layers 

## Firstly imports packages and functions, and then goes on to configure the map itself.


In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
import polyline
from collections import defaultdict
from strava_analysis import create_heatmap_data, decode_polyline, create_base_map, add_heatmap_layers, add_legend, add_layer_control

## Call to the function and create heat map

This section generates an interactive heatmap of the Strava activities using the create_heatmap function. The function reads activity data from a CSV file, decodes the GPS polylines, and computes two types of heatmap layers: one showing the frequency of activity across routes, and another visualizing the average speed along those routes.

The map is currently configured to center over Salzburg, and users can toggle between the speed and  in the map layers.

The final interactive map is saved as an HTML.

In [ ]:

def create_heatmap(csv_file, polyline_column, speed_column, center, zoom, output_file):
    """
    Main function to create the complete heatmap
    """
    # Load data 
    print("Loading CSV...")
    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} activities from CSV")
    
    # Process coordinates (using existing create_heatmap_data function)  
    freq_coords, speed_coords = create_heatmap_data(df, polyline_column, speed_column)
    
    if not freq_coords and not speed_coords:
        print("No valid coordinates found. Check your CSV file and columns.")
        return None
    
    # Create map with all components
    m = create_base_map(center, zoom)
    m = add_heatmap_layers(m, freq_coords, speed_coords)
    m = add_legend(m)
    m = add_layer_control(m)
    
    # Save map
    m.save(output_file)
    print(f"Heatmap saved as: {output_file}")
    return m

# Configuration
CSV_FILE = "strava_data.csv"
POLYLINE_COLUMN = 'map.summary_polyline'
SPEED_COLUMN = 'average_speed'
SALZBURG_CENTER = (47.8095, 13.0550)
ZOOM_LEVEL = 10
OUTPUT_FILE = 'strava_salzburg_heatmap.html'

create_heatmap(CSV_FILE, POLYLINE_COLUMN, SPEED_COLUMN, SALZBURG_CENTER, ZOOM_LEVEL, OUTPUT_FILE)